In [1]:
#importing libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import functools
import math
import livelossplot
import pickle
from Utils.helper import *

In [6]:
print("Simulating First Match")
match_sim2 = Match(CSK_Squad,MI_Squad,MI_Pitch,Display=1)

Simulating First Match
Mumbai Indians won the toss and chose to Bowl first
Chennai Super Kings  :  167 / 10  in  13 . 2
Extras: 3



,Batsman,Runs,Fours,Sixes,Balls Faced,Dissmissal Type,Dismissed By
0,F du Plessis,41,4,2,21,Caught,Harbhajan Singh
1,RV Uthappa,9,2,0,5,Caught,Harbhajan Singh
2,SK Raina,9,0,1,6,Caught,JJ Bumrah
3,AT Rayudu,12,1,1,5,Caught,TG Southee
4,KM Jadhav,10,0,1,6,Bowled,MJ McClenaghan
5,MS Dhoni,27,2,2,10,Bowled,TG Southee
6,RA Jadeja,8,1,0,6,Caught,JJ Bumrah
7,DJ Bravo,24,5,0,10,Not Out,-
8,KV Sharma,12,1,1,5,Caught,Harbhajan Singh
9,DL Chahar,12,1,1,5,Bowled,JJ Bumrah


,Bowler,Runs Conceded,Wickets Taken,Overs,Batsman Names
0,TG Southee,40,2,3.0,"AT Rayudu, MS Dhoni"
1,Harbhajan Singh,31,3,3.0,"RV Uthappa, F du Plessis, KV Sharma"
2,MJ McClenaghan,35,1,3.0,KM Jadhav
3,JJ Bumrah,30,4,2.2,"SK Raina, RA Jadeja, DL Chahar, SN Thakur"
4,SL Malinga,30,0,2.0,-


,Dismissed Batsman,Team Runs,Overs
1,RV Uthappa,12,2
2,SK Raina,33,4
3,AT Rayudu,60,6
0,F du Plessis,81,7
4,KM Jadhav,82,8
6,RA Jadeja,99,9
5,MS Dhoni,127,11
7,KV Sharma,145,12
8,DL Chahar,166,14
9,SN Thakur,167,14


,Over,Bowler,Runs Conceded,Wickets Taken,Total Score,Total Wickets
0,1,TG Southee,11,0,11,0
1,2,Harbhajan Singh,4,1,15,1
2,3,MJ McClenaghan,11,0,26,1
3,4,JJ Bumrah,19,1,45,2
4,5,SL Malinga,14,0,59,2
5,6,TG Southee,13,1,72,3
6,7,Harbhajan Singh,10,1,82,4
7,8,MJ McClenaghan,8,1,90,5
8,9,JJ Bumrah,10,1,100,6
9,10,SL Malinga,16,0,116,6


...

,Batsman,Runs,Fours,Sixes,Balls Faced,Dissmissal Type,Dismissed By
0,PA Patel,34,5,1,29,Run Out,-
1,JC Buttler,0,0,0,10,Caught,RA Jadeja
2,RG Sharma,47,6,1,40,Run Out,-
3,HH Pandya,8,2,0,4,Run Out,-
4,KA Pollard,9,1,0,6,Caught,SN Thakur
5,KH Pandya,4,0,0,6,Caught,DL Chahar
6,Harbhajan Singh,10,2,0,7,Caught,KV Sharma
7,MJ McClenaghan,19,1,2,6,Bowled,KV Sharma
8,TG Southee,1,0,0,2,Stumped,RA Jadeja
9,JJ Bumrah,2,0,0,7,Not Out,-


,Bowler,Runs Conceded,Wickets Taken,Overs,Batsman Names
0,DL Chahar,25,1,4.0,KH Pandya
1,SN Thakur,39,1,4.0,KA Pollard
2,KV Sharma,36,2,4.0,"MJ McClenaghan, Harbhajan Singh"
3,RA Jadeja,35,2,4.0,"JC Buttler, TG Southee"
4,DJ Bravo,23,0,4.0,-


,Dismissed Batsman,Team Runs,Overs
1,JC Buttler,16,4
0,PA Patel,48,9
3,HH Pandya,59,10
4,KA Pollard,85,12
2,RG Sharma,113,16
5,KH Pandya,117,16
7,MJ McClenaghan,141,18
6,Harbhajan Singh,146,18
8,TG Southee,152,19


,Over,Bowler,Runs Conceded,Wickets Taken,Total Score,Total Wickets
0,1,DL Chahar,1,0,1,0
1,2,SN Thakur,0,0,1,0
2,3,KV Sharma,0,0,1,0
3,4,RA Jadeja,16,1,17,1
4,5,DJ Bravo,2,0,19,1
5,6,DL Chahar,5,0,24,1
6,7,SN Thakur,10,0,34,1
7,8,KV Sharma,14,0,48,1
8,9,RA Jadeja,8,1,56,2
9,10,DJ Bravo,4,1,60,3


Chennai Super Kings won by 4 Runs


In [2]:
import numpy as np
np.array([[1],[2],[3]]).shape

(3, 1)